#  Optimal Restaurant Location Finder Toronto 


In [21]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
%matplotlib inline 
import matplotlib.cm as cm
import matplotlib.colors as colors

## First we will be using BeautifulSoup to scrape wikipedia for neighbourhood information, including Postal Code Borough, and Neighborhood. The information is to be stored in dataframe df_Toronto.

In [22]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page.content;
soup= BeautifulSoup(page.content,"xml")
print(soup.prettify());
table= soup.find('table',{'class':'wikitable sortable'})
table_rows=table.findAll('tr')
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])
df_Toronto= pd.DataFrame(data, columns=['Postal Code','Borough','Neighbourhood'])
df_Toronto=df_Toronto[~df_Toronto['Postal Code'].isnull()]


<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="UTF-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"449ccf20-e1f6-4d16-813e-c13126a76ab2","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":969510799,"wgRevisionId":969510799,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in O


## Clean data by removeing "Not assigned", any "Not assigned" in Neighbourhood will be replaced with Borough value

In [23]:
df_Toronto.head()

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [24]:
df_Toronto= df_Toronto[df_Toronto.Borough!="Not assigned"]


In [25]:
df_Toronto

,Postal Code,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
161,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
166,M4Y,Downtown Toronto,Church and Wellesley
169,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
170,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [26]:
df_Toronto.Neighbourhood.replace("Not assigned",df_Toronto.Borough,inplace=True)
df_Toronto.shape

(103, 3)

## Obtain Latitude and Longitude from csv and merge to original Tlocation_df


In [27]:
df_lat_long= pd.read_csv(r'C:\Users\BK\Desktop\projects\Coursera_Capstone\KMeans\Geospatial_Coordinates.csv')
df_lat_long.isna().sum()
df_lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
Tlocation_df= df_Toronto.merge(df_lat_long,on= "Postal Code", how = "left")
Tlocation_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [29]:
Tlocation_df.isna().sum()

Postal Code      0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

In [30]:
import folium
from sklearn.cluster import KMeans

## Use geolocator to convert latitude and longitude, and use Folium visualize the neighbourhood on the map

In [31]:


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ON_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))



Solving environment: ...working... done

# All requested packages already installed.

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [32]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
Limit=100
for lat, lng, borough, neighborhood in zip(Tlocation_df['Latitude'], Tlocation_df['Longitude'], Tlocation_df['Borough'], Tlocation_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


## In this section we will be getting nearby venues for each neighborhood along with the type of venue using the FourSquare API

In [33]:
import json

with open('Secretkey.json') as f:
    data = json.load(f)
    CLIENT_SECRET = data['Secretkey']

In [34]:
CLIENT_ID = 'LAH4R3HEEGAT000NXVQVWZQOQN111D0FEABC1GJRJIFWS5AK' 
VERSION = '20180605' # Foursquare API version

In [35]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:

Toronto_venues = getNearbyVenues(names=Tlocation_df['Neighbourhood'],
                                   latitudes=Tlocation_df['Latitude'],
                                   longitudes=Tlocation_df['Longitude']
                                  )

Parkwoods


KeyError: 'groups'

##  Create dummy variable for each venue and get the mean frequency per venue, then merge to to get associated neighborhoods.


In [ ]:


# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()



In [ ]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped



## Filter the venues to only restaurants

In [ ]:
Restaurant=Toronto_grouped.filter(like='Restaurant').columns
Toronto_Data=Toronto_grouped.loc[:,(Restaurant)]
Toronto_Data=Toronto_Data.set_index(Toronto_grouped['Neighborhood'])
Toronto_Data
Toronto_Data.reset_index(inplace=True)
Toronto_Data.head()


## Allow User to input what type of restaurant they want to open

In [ ]:
Type= input() + " Restaurant"

# Type= "Sushi Restaurant"


## Review Data:Mean Frequencies, and top 5 Neighbourhoods

In [ ]:
Restaurants=Toronto_Data.loc[:,Type]
Restaurants.max()

In [ ]:
new= Toronto_Data[["Neighborhood",Type]]
new.sort_values(by=Type, ascending=False).head()


## Apply Unsupervised Machine Learning Clustering Algorithm, KMeans. Find Optimal K using Elbow Point.

In [ ]:
Toronto_Data_K=Toronto_Data.drop('Neighborhood', 1)
distortions = []
K = range(1,30)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(Toronto_Data_K)
    distortions.append(kmeanModel.inertia_)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
# set number of clusters
kclusters = 6


kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_Data_K)

## Reviewing Data Again

In [ ]:
Toronto_Data_K.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_Data_K.insert(0, 'Neighborhood', Toronto_Data.Neighborhood)

Toronto_Data_K.head()


In [ ]:
Toronto_Data.head()
Toronto_Data.isna().sum()

In [ ]:

Toronto_Data_K.reset_index(inplace=True)
Toronto_merged=Tlocation_df
Toronto_merged = Toronto_merged.join(Toronto_Data_K.set_index('Neighborhood'), on='Neighbourhood')


In [ ]:
Toronto_merged.isna().sum()
Toronto_merged.head(20)

## Visualize Map Clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_Data_K['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Finding which cluster has the highest Mean Frequency & choosing optimal cluster accordingly

In [ ]:

clusters= Toronto_Data_K[['Neighborhood','Cluster Labels',Type]]
clusters = clusters.iloc[:,1:].groupby('Cluster Labels').sum().transpose()
clusters.head()


In [ ]:
Cluster0=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0,Toronto_merged.columns[[2,3,4] + list(range(6, Toronto_merged.shape[1]))]]
Cluster0.reset_index(inplace = True) 
Cluster0.drop(columns=['index'],inplace=True)

Cluster0

In [ ]:
Neigh_list = Cluster0['Neighbourhood'].tolist()


## Get nearby venues for neighbourhoods in cluster 0

In [ ]:
def getNearbyVenues2(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:

Cluster_venues_= getNearbyVenues2(names=Cluster0.Neighbourhood,
                                   latitudes=Cluster0.Latitude,
                                   longitudes=Cluster0.Longitude
                                  )


## Counting  all of the different restaurants that are visited in each neighborhood. Using this to compare the demand(Frequency in Neighbourhood) with supply(Different Restaurants in Neighbourhood)

In [ ]:
Venues=Cluster_venues_.loc[Cluster_venues_['Venue Category'] == "Sushi Restaurant"]
Venues.head()

Cluster_venues_type=Venues
Cluster_venues_type=Cluster_venues_type.Neighborhood.value_counts()
Cluster_venues_type=Cluster_venues_type.to_frame()
Cluster_venues_type.reset_index( inplace=True)


In [ ]:
Cluster_venues_type.columns=["Neighbourhood", "visits per Neighbourhood"]
Cluster_venues_type

In [ ]:
Freq.info()

In [ ]:
Freq=Cluster0.loc[:,("Neighbourhood",Type)]
Freq=Freq[Freq[Type]>0]
Freq



In [ ]:
Cluster_venues_type=Cluster_venues_type.merge(Freq, on='Neighbourhood')
Cluster_venues_type.drop_duplicates(subset ="Neighbourhood", keep = 'first', inplace = True) 


## Sorting Based on unique visits per neighbourhood and frequency of Sushi Restaurant

In [ ]:
Cluster_venues_type.sort_values(["visits per Neighbourhood",Type], ascending=[True,False],inplace=True)
Cluster_venues_type.reset_index(inplace=True)
del Cluster_venues_type['index']
Cluster_venues_type

## Visualize Optimal Location Using Bar Chart,  using Indexes instead of neighbourhood name

In [ ]:
fig = plt.figure(figsize=(30,10))
x=Cluster_venues_type.index
temp="visits per Neighbourhood"
y=Cluster_venues_type[Type]/Cluster_venues_type[temp]

x_pos = [i for i, _ in enumerate(x)]

plt.bar(x_pos, y, color='green')
plt.xlabel("Neighbourhood")
plt.ylabel("Total Visits/Frequency")
plt.title("Optimal Neighbourhood")

plt.xticks(x_pos, x)
plt.show()